In [1]:
import numpy as np

def reweight_distribution(original_distribution, temperature = 0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distibution)

In [16]:
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', 
                           origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path, encoding="utf-8").read().lower()
print('Corpus length: ', len(text))

Corpus length:  600893


In [26]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences: ', len(sentences))

Number of sequences:  200278


In [27]:
chars = sorted(list(set(text)))
print('Unique characters: ', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)

Unique characters:  57
{'g': 33, 'é': 55, 'l': 38, 'r': 44, 'e': 31, 'u': 47, '!': 2, 'w': 49, '3': 13, 't': 46, 'q': 43, '9': 19, "'": 4, '5': 15, 'æ': 54, '7': 17, 'd': 30, '=': 22, 'y': 51, '"': 3, 'b': 28, '1': 11, '4': 14, 'n': 40, '(': 5, 'h': 34, '-': 8, 'c': 29, 'o': 41, 'm': 39, '8': 18, '?': 23, 'ä': 53, 's': 45, 'f': 32, '2': 12, ')': 6, '\n': 0, ' ': 1, 'z': 52, 'x': 50, '[': 24, ':': 20, 'v': 48, ']': 25, ',': 7, '6': 16, 'k': 37, 'j': 36, 'i': 35, '0': 10, 'ë': 56, 'a': 27, '.': 9, ';': 21, '_': 26, 'p': 42}


In [28]:
print('Vectorization . . . ')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization . . . 


In [29]:
print(x.shape)

(200278, 60, 57)


In [30]:
print(y.shape)

(200278, 57)


In [33]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = 'softmax'))

optimizer = keras.optimizers.RMSprop(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', 
              optimizer = optimizer)

In [34]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [37]:
import random
import sys

for epoch in range(1, 60):
    print('Epoch: ', epoch)
    
    model.fit(x, y, batch_size = 128, epochs = 1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('---> temperature: ', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose = 0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)

Epoch:  1
Epoch 1/1
200278/200278 [==============================] - 83s 416us/step - loss: 1.5527
--- Generating with seed: "ns, fears,
exhaustions and delights of the saint are well kn"
---> temperature:  0.2
ns, fears,
exhaustions and delights of the saint are well knows and suppression of the present things and things and the man which is a man a still self-spirit of the moral the will be the will the spirit, and present and stronger and things of the strengthers of the strengthers of the desire of the be the suppose that is a man and in the present things of the will the will the things and stronged the strengthers and self--a prince of the strength of the p---> temperature:  0.5
 the strengthers and self--a prince of the strength of the pasist the desirity and latter of the feel of this interrection of the strength be a misperience, to be all that the world of things of the spirit cultary and there are words a desire of christianity in its things in this enuthers prouptring and m

jucons in listure thos rilliwhte from the virt "them are world toforne by this worght and by qualqu has disundly
their more desranges,"
going the sensus in this virtue without a is forstance of
new de-can pleght to the wrond
effection
as he has power, renltery, judgt.

live will were your home
hanowed to ina oEpoch:  5
Epoch 1/1
200278/200278 [==============================] - 80s 399us/step - loss: 1.4383
--- Generating with seed: "-is among the means whereby the burden of life can be made
l"
---> temperature:  0.2
-is among the means whereby the burden of life can be made
like the contemplation of the greatest the spirit in the consequently and soul and problem of the spirit of the self--as a problem of the greatest of the desires and soul and there is the spirit and soul and conditions and than the spirit and in the desires and refined the spirit the stronger and than the spirit of the same the continue and and the spirit of the spirit and and than the spirit that ---> temperature: 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


uority more still?--powe proffath uncersalistic is responsibles that replanntiantmined. he who wance pain onne for
would "the words." must knows good. there even wither one of concernalmy condess. he does adain distruastic to laigh has as one's proffurise benual"), when there is he was ho---> temperature:  1.2
gh has as one's proffurise benual"), when there is he was honeel and will to expense babinus
philosopher purstway curie to hears almostblationk that dangeread
"itodistorway, does loned.j? prationtful dleping highy to odle a may, and gives theologianking; he fenls? hostery
or orequisus becousicless un-itself, characy aims.paily: these precrsys, become hiv notuhe, who shondly: towarler momence of misundewstation, rebeinial . 
, parties and inewingoliy and boEpoch:  6
Epoch 1/1
200278/200278 [==============================] - 83s 413us/step - loss: 1.4249
--- Generating with seed: "ae veritates,
he evinced the very pride with which he raised"
---> temperature:  0.2
ae veritates,
he 

completed to makes the former and and the most still us in the hearting of the strength and persived and as the belief are spirits the only the principle, and conscience of th---> temperature:  1.0
ief are spirits the only the principle, and conscience of them
to disguite it for exeny: he goltal of life a fele how make
it-dole-so perplussable the schate are comeas the gives the free world for instance
of himrers! shar
slarncean. not enloned  pessions our one and sympathing which here the depart a madistings of himself, onner philosopher, while learning, world.
the ss.

veeked, as one signity and this befinity
in
stands.

inesel, grishesial: it is fol---> temperature:  1.2
 and this befinity
in
stands.

inesel, grishesial: it is folly; a will nor fimple
doen, that it was san
 let mis, in
the world and some world in napare gas slorm--u! attainbly, auran the ediour common my revolxuply, and the hado!
lut of
order would simple of lack
of hhmers drendfus, the triuth.

preaiture," concerning

of the world in the contrary of the more more more souls of the seasism, and the things
prefessed therefore, in the favourest, and indemnetical encliness. it is most down, and nothing of the philosophers, which is whole have no example and interpretation of an interestion of the more influence of conceive the habie just that a are instinct of the fair, whose more the
spirit to the more has the conception of every not of the contrary of an included in the w---> temperature:  1.0
ception of every not of the contrary of an included in the wors or their hoque? it is the alterstofder things.
there is gercippefar, was these evad. they, the as the firstith; under she still haditatical deraring, can resepticion in what is free fstrendomazed, (that have ,oeg, socience
of
the fibert. matace of the most
dragai, it is in the last goders to a would not
lostning
ref
interestical, the home of fromesoral schilownaus, under general also invention---> temperature:  1.2
home of fromesoral schilownaus, un

actually and sense to the present and so is a discovered and the explanation of the spirit and something of the same the present to the exceptionable of the same the probables and the community of the spirit and the spirit to the same to the most such a such a spirit of the same the world of the same the problem of the communition of the same and the excell in the exceptionable to the same to the comm---> temperature:  0.5
 and the excell in the exceptionable to the same to the command and refined problem of the predectable
doobes to a baring the unconscience of a most communition of the moral nature of the wicked and such a sickrition of the deedding, and
when a galable young and has the counter souls, in the great desired the exprassion, is a profound as the commands, of the present, precisely to have all command
as the man is the reason of the charmity believes to the rea---> temperature:  1.0
as the man is the reason of the charmity believes to the readade" to of the gon-wretchessi

KeyboardInterrupt: 